In [ ]:
# Importing Libraries

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Graphing libraries 
import matplotlib.pyplot as plt
import seaborn as sns

# Importing all soup to nuts libraries
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.pipeline import Pipeline, make_pipeline
import lightgbm as lgb

# Importing regression boosting
from xgboost import XGBRegressor
from catboost import CatBoostRegressor



In [ ]:
# Import data

train_dataframe = pd.read_csv('data/train.csv')
test_dataframe = pd.read_csv('data/test.csv')

# Describing train data
train_dataframe.describe()

In [ ]:
# Printing out dataframe columns

train_dataframe.columns

In [ ]:
# Looking for all possible numerical columns that we can utilize
# Now we'll start preprocesssing data

train_dataframe.dtypes[train_dataframe.dtypes != 'object']

In [ ]:
# Next we're going to compare different attributes to sales price to see the positive/negative correlation 
# Starting with lot frontage 
# Below is a graph comparing Lot Frontage to Sales Price


plt.scatter(x='LotFrontage', y='SalePrice', data=train_dataframe)


In [ ]:
##There are a few outliers to the right but overall we can see a positive correlation between LotFrontage and SalePrice
# Lets grab id of the two outlier plotpoints so we can remove them to obtain more cohesive data

train_dataframe.query('LotFrontage > 300')

# We're going to remove 935 and 1299

In [ ]:
# Continuing with the trend above we're going to continue comparisons to sales price and also continue removing outliers
# Next is overall quality, we're going to compare OverallQual to SalePrice 

plt.scatter(x='OverallQual', y='SalePrice', data=train_dataframe)



In [ ]:
# As we can see OverallQual has a postive correlation with SalePrice
# There's a outlier in the end but I believe we can keep it
# Next we will do lot area 

plt.scatter(x='LotArea', y='SalePrice', data=train_dataframe)

In [ ]:
# There are four outliers that I would like to remove

train_dataframe.query('LotArea > 55000')
# Looking to drop 250, 314, 336, 707


In [ ]:
# The next value is year built

plt.scatter(x='YearBuilt', y='SalePrice', data=train_dataframe)

# No outliers I want to remove 

In [ ]:
# Overall condition comparison

plt.scatter(x='OverallCond', y='SalePrice', data=train_dataframe)

# There is one outlier in column two I want to remove
# 379
train_dataframe.query("OverallCond==2")


In [ ]:
# Year Built 

plt.scatter(x='YearBuilt', y='SalePrice', data=train_dataframe)

In [ ]:
# Removing a single outlier house that was built before 1900 and is almost 500k
# We will be repeating this process for every numerical attribute. Comments will be less common to better streamline the project
train_dataframe.query('YearBuilt < 1900 & SalePrice > 400000')
# Remove: 186

In [ ]:
# Year Modeling date

plt.scatter(x='YearRemodAdd', y='SalePrice', data=train_dataframe)

In [ ]:
# No outliers to drop above

# Masonry veneer type
plt.scatter(x='MasVnrArea', y='SalePrice', data=train_dataframe)

In [ ]:
train_dataframe.query('MasVnrArea > 1350')

# Removing:
# 298
# 1170

In [ ]:
# BsmtFinSF1: Type 1 finished square feet
plt.scatter(x='BsmtFinSF1', y='SalePrice', data=train_dataframe)


In [ ]:
# One outlier to the right with high amount of finished square feet and low price
train_dataframe.query('BsmtFinSF1 > 5000')

# Remove: 1299


In [ ]:
# BsmtFinSF1: Type 1 finished square feet
plt.scatter(x='BsmtFinSF2', y='SalePrice', data=train_dataframe)

In [ ]:
# One outlier in between the 400 - 600 column
train_dataframe.query('BsmtFinSF2 > 400 & SalePrice > 500000')

# drop: 441

In [ ]:
# BsmtUnfSF: Unfinished square feet of basement area
# No outliers on this one
plt.scatter(x='BsmtUnfSF', y='SalePrice', data=train_dataframe)

In [ ]:
# TotalBsmtSF: Total square feet of basement area
plt.scatter(x='TotalBsmtSF', y='SalePrice', data=train_dataframe)

In [ ]:
# One outlier
train_dataframe.query('TotalBsmtSF > 5000')

# Drop: 1299

In [ ]:
# 1stFlrSF: First Floor square feet
# No outliers
plt.scatter(x='1stFlrSF', y='SalePrice', data=train_dataframe)

In [ ]:
# 2ndFlrSF: Second Floor square feet
# No outliers
plt.scatter(x='2ndFlrSF', y='SalePrice', data=train_dataframe)


In [ ]:
# LowQualFinSF: Low quality finished square feet
plt.scatter(x='LowQualFinSF', y='SalePrice', data=train_dataframe)


In [ ]:
# One outlier to the right
train_dataframe.query('LowQualFinSF > 500')

# Drop: 186

In [ ]:
# BsmtFullBath: Basement full bathrooms
plt.scatter(x='BsmtFullBath', y='SalePrice', data=train_dataframe)

In [ ]:
# One outlier to the right
train_dataframe.query('BsmtFullBath == 3')

# Drop: 739


In [ ]:
# GrLivArea: Above grade (ground) living area square feet
plt.scatter(x='GrLivArea', y='SalePrice', data=train_dataframe)

In [ ]:
# We see two outliers to the right
train_dataframe.query('GrLivArea > 4400')

# Drop 524, 1299


In [ ]:
#BsmtFullBath: Basement half bathrooms
plt.scatter(x='BsmtHalfBath', y='SalePrice', data=train_dataframe)

In [ ]:
# One outlier with the 3 full baths and a low price
train_dataframe.query('BsmtFullBath == 2')

# Drop 598 and 955

In [ ]:
# FullBath: Full bathrooms above grade
# No outliers
plt.scatter(x='FullBath', y='SalePrice', data=train_dataframe)


In [ ]:
# HalfBath: Half baths above grade
#No outliers
plt.scatter(x='HalfBath', y='SalePrice', data=train_dataframe)

In [ ]:
# KitchenAbvGr Kitchens above grade
plt.scatter(x='KitchenAbvGr', y='SalePrice', data=train_dataframe)

In [ ]:
# Two outliers with 3 kitchens and low sales price
train_dataframe.query('KitchenAbvGr == 3')

#Drop: 49 and 810

In [ ]:
# BedroomAbvGr: bedrooms above grade
plt.scatter(x='BedroomAbvGr', y='SalePrice', data=train_dataframe)

In [ ]:
# One outlier with 8 bedrooms
train_dataframe.query('BedroomAbvGr == 8')

# Drop 636

In [ ]:
# TotRmsAbvGrd: Total rooms above grade

plt.scatter(x='TotRmsAbvGrd', y='SalePrice', data=train_dataframe)



In [ ]:
# An outlier to the right with 14 rooms and low price

train_dataframe.query('TotRmsAbvGrd == 14')

# Drop: 636

In [ ]:
# GarageYrBlt: Year that garage was built
# No outliers
plt.scatter(x='GarageYrBlt', y='SalePrice', data=train_dataframe)

In [ ]:
#GarageCars: Number of cars that Garage can hold
# No outliers
plt.scatter(x='GarageCars', y='SalePrice', data=train_dataframe)

In [ ]:
# GarageArea: Area of garage 

plt.scatter(x='GarageArea', y='SalePrice', data=train_dataframe)

In [ ]:
# Two outliers with large garage area and low sales price 
train_dataframe.query('GarageArea > 1200')

# Drop 1062, 1191

In [ ]:
# Fireplaces: Number of fireplaces
# No outliers
plt.scatter(x='Fireplaces', y='SalePrice', data=train_dataframe)

In [ ]:
# WoodDeckSF: Wood deck area in square feet
plt.scatter(x='WoodDeckSF', y='SalePrice', data=train_dataframe)

In [ ]:
# OpenPorchSF: OpenPorchSF: Open porch area in square feet
plt.scatter(x='OpenPorchSF', y='SalePrice', data=train_dataframe)

In [ ]:
# One outlier with high OpenPorchSF and low SalePrice

train_dataframe.query('OpenPorchSF > 500')

# Drop: 496

In [ ]:
# Enclosed Porch Area
plt.scatter(x='EnclosedPorch', y='SalePrice', data=train_dataframe)

In [ ]:
# One outlier 
train_dataframe.query('EnclosedPorch > 500')

# Drop: 198

In [ ]:
#ScreenPorch: Screen porch area in square feet
plt.scatter(x='ScreenPorch', y='SalePrice', data=train_dataframe)

In [ ]:
# 3SsnPorch: Three season porch area in square feet
plt.scatter(x='3SsnPorch', y='SalePrice', data=train_dataframe)


In [ ]:
# EnclosedPorch: Enclosed porch area in square feet
plt.scatter(x='PoolArea', y='SalePrice', data=train_dataframe)

In [ ]:
# Dropping outliers and creating training dataframe
dropped_values = [935, 1299, 250, 314, 336, 707, 379, 186, 298, 1170, 441, 739, 524, 598, 955, 49, 810, 636, 1062, 1191]

train_dataframe = train_dataframe[train_dataframe.Id.isin(dropped_values) == False]


In [ ]:
# Displaying Null Values per column and deciding which columns to drop
pd.DataFrame(train_dataframe.isnull().sum().sort_values(ascending=False)).head(20)

In [ ]:
# Dropping Columns with heavy amounts of missing values from both training and test dataframes or attributes that are irrelevant to our model

train_dataframe = train_dataframe.drop(columns=['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'GarageYrBlt', 'GarageCond', 'BsmtFinType2'])
train_dataframe.columns


In [ ]:
test_dataframe = test_dataframe.drop(columns=['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'GarageYrBlt', 'GarageCond', 'BsmtFinType2'])
test_dataframe.columns

In [ ]:
# Taking a look at columns with NaN values that can be filled in or changed in order to give us data we use in our model
# Starting with MasVnrType

train_dataframe['MasVnrType'].unique() 

# We're going to replace the NaN type with a "No" value

train_dataframe['MasVnrType'].fillna('No', inplace=True)
test_dataframe['MasVnrType'].fillna('No', inplace=True)


In [ ]:
# Creating a boxplot for MasVnrType to see how the attribute changed after value substitution
sns.catplot(data=train_dataframe, x="MasVnrType", y="SalePrice", kind="box")

In [ ]:
# Filling in numerical values with zero for MasVnrArea
train_dataframe['MasVnrArea'].fillna(0, inplace=True)
test_dataframe['MasVnrArea'].fillna(0, inplace=True)

In [ ]:
# Filling in nan values for FirePlaceQu
train_dataframe['FireplaceQu'].unique()

train_dataframe['FireplaceQu'].fillna('No', inplace=True)
test_dataframe['FireplaceQu'].fillna('No', inplace=True)

# For BsmtFinType1 we're filling in the nan values with unfinished attribute instead
train_dataframe['BsmtFinType1'].fillna('Unf', inplace=True)
test_dataframe['BsmtFinType1'].fillna('Unf', inplace=True)


# Filling in values for LotFrontage
train_dataframe['LotFrontage'].fillna(0, inplace=True)
test_dataframe['LotFrontage'].fillna(0, inplace=True)

# Filling in values for GarageType and GarageQuality 
train_dataframe['GarageType'].fillna('No', inplace=True)
test_dataframe['GarageType'].fillna('No', inplace=True)

# Changing the nan values in BsmtExposure to no
train_dataframe['BsmtExposure'].fillna('No', inplace=True)
test_dataframe['BsmtExposure'].fillna('No', inplace=True)

# For the next attribute, BsmtQual is actually the height of the basement
train_dataframe['BsmtQual'].fillna('No', inplace=True)
test_dataframe['BsmtQual'].fillna('No', inplace=True)

train_dataframe['BsmtCond'].fillna('No', inplace=True)
test_dataframe['BsmtCond'].fillna('No', inplace=True)

# For the electrical attribute we're going to fill the nan value with SBrk which is the standard circuit and breaker

train_dataframe['Electrical'].fillna('SBrkr', inplace=True)
test_dataframe['Electrical'].fillna('SBrkr', inplace=True)


In [ ]:
# Next I'd like to create some features to include in our model

# Total Square Feet 
train_dataframe['TotalSF'] = train_dataframe['1stFlrSF'] + train_dataframe['2ndFlrSF'] + train_dataframe['BsmtFinSF1'] + train_dataframe['BsmtFinSF2']
test_dataframe['TotalSF'] = test_dataframe['1stFlrSF'] + test_dataframe['2ndFlrSF'] + test_dataframe['BsmtFinSF1'] + test_dataframe['BsmtFinSF2']

# Total Area 
train_dataframe['TotalArea'] = train_dataframe['TotalBsmtSF'] + train_dataframe['GrLivArea'] 
test_dataframe['TotalArea'] = test_dataframe['TotalBsmtSF'] + test_dataframe['GrLivArea'] 

# Age of the house 
train_dataframe['HouseAge'] = train_dataframe['YrSold'] - train_dataframe['YearBuilt']
test_dataframe['HouseAge'] = test_dataframe['YrSold'] - test_dataframe['YearBuilt']

# Total number of baths
train_dataframe['TotalBaths'] = train_dataframe['FullBath'] + train_dataframe['BsmtFullBath'] + (0.5 * (train_dataframe['HalfBath'] + train_dataframe['BsmtHalfBath']))
test_dataframe['TotalBaths'] = test_dataframe['FullBath'] + test_dataframe['BsmtFullBath'] + (0.5 * (test_dataframe['HalfBath'] + test_dataframe['BsmtHalfBath']))

# How old the remodeling is, age of remodel
train_dataframe['HouseRemodAge'] = train_dataframe['YrSold'] - train_dataframe['YearRemodAdd']
test_dataframe['HouseRemodAge'] = test_dataframe['YrSold'] - test_dataframe['YearRemodAdd']

# Total Porch Squarefeet 
train_dataframe['TotalPorchSF'] = train_dataframe['OpenPorchSF'] + train_dataframe['WoodDeckSF']  + train_dataframe['ScreenPorch']  + train_dataframe['3SsnPorch'] + train_dataframe['EnclosedPorch']
test_dataframe['TotalPorchSF'] = test_dataframe['OpenPorchSF'] + test_dataframe['WoodDeckSF']  + test_dataframe['ScreenPorch']  + test_dataframe['3SsnPorch'] + test_dataframe['EnclosedPorch']



In [ ]:
# Dropping all the unnecessary columns and columns that were used to create our new features

train_dataframe = train_dataframe.drop(columns=['Id','BsmtFinSF1', 'BsmtFinSF2', 'GrLivArea', 'TotalBsmtSF','BsmtFullBath', 'FullBath', 'BsmtHalfBath','YrSold', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'OpenPorchSF', '3SsnPorch', 'EnclosedPorch', 'ScreenPorch','WoodDeckSF', 'HalfBath'])
test_dataframe = test_dataframe.drop(columns=['BsmtFinSF1', 'BsmtFinSF2', 'GrLivArea', 'TotalBsmtSF','BsmtFullBath', 'FullBath', 'BsmtHalfBath','YrSold', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'OpenPorchSF', '3SsnPorch', 'EnclosedPorch', 'ScreenPorch','WoodDeckSF', 'HalfBath'])


In [ ]:
# Creating a correlation graph between numeric attributes

correlation = train_dataframe.corr(numeric_only=True) 
plt.figure(figsize=(22,14))
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt=".3f")

# Dropping GarageArea because of the high correlation of GarageCars
train_dataframe = train_dataframe.drop(columns=['GarageArea'])
test_dataframe = test_dataframe.drop(columns=['GarageArea'])


In [ ]:
# Creating a histogram to show the number of houses at each pricepoint

# Running a log through Saleprice to get a better blanced histogram
train_dataframe['SalePrice'] = np.log1p(train_dataframe['SalePrice'])

sns.histplot(train_dataframe, x=train_dataframe['SalePrice'])

In [ ]:
# We have to take the columns that are objects and use ordinal encoding or one hot encoding to include them
train_dataframe.dtypes[train_dataframe.dtypes=='object']

In [ ]:
# Categorizing columns between ordinal coding and OHE.
# We're also going to prep our numerical column

num_cols = train_dataframe.select_dtypes(include=['int64', 'float64']).columns

# Dropping SalePrice 
num_cols = num_cols.drop('SalePrice')

# Rule of thumb is if order of value matters then we put the object attribute under ordinal

ordinal_cols = ['GarageFinish', 'HeatingQC', 'ExterQual', 'GarageQual', 'BsmtCond', 'BsmtExposure', 'KitchenQual', 'Functional', 'ExterCond', 'PavedDrive', 'LotShape', 'LandContour', 'BsmtQual', 'BsmtFinType1','Utilities', 'CentralAir','LandSlope', 'FireplaceQu']

hot_cols = ['Street','Exterior1st', 'Exterior2nd','Condition1', 'Condition2', 'HouseStyle', 'RoofStyle', 'LotConfig', 'Neighborhood','Heating', 'BldgType','GarageType', 'RoofMatl', 'Electrical','SaleType', 'MSZoning', 'SaleCondition', 'Foundation','MasVnrType']

train_dataframe.dtypes[train_dataframe.dtypes != 'object']

In [ ]:
# Going to create the three model pipelines
# Pipelines will go as follows:
# impute -> Ordinal Encoder
# impute -> OneHotEncoder
# impute -> scaler

ordinal_pipeline = Pipeline(steps=[('impute', SimpleImputer(strategy='most_frequent')), ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))])
hot_pipeline = Pipeline(steps=[('impute', SimpleImputer(strategy='most_frequent')), ('hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])
num_pipeline = Pipeline(steps=[('impute', SimpleImputer(strategy='mean')),('scaler', StandardScaler())])

In [ ]:
# Creating a Column transformer
column_transformer = ColumnTransformer(transformers=[('ordinal_pipe', ordinal_pipeline, ordinal_cols), ('hot_pipe', hot_pipeline, hot_cols), ('num_pipe', num_pipeline, num_cols)], remainder='passthrough', n_jobs=-1)

# Preprocessing our pipeline
preprocess_pipeline = Pipeline(steps=[('preprocessing', column_transformer)])

# Creating X and Y coordinates 

x = train_dataframe.drop('SalePrice', axis=1)
y = train_dataframe['SalePrice']

# Preprocessing x
x_preprocess = preprocess_pipeline.fit_transform(x)

#Splitting training Data
# Used a traditional 80/20 split with a random state of 30
X_train, X_test, y_train, y_test = train_test_split(x_preprocess, y, test_size=0.2, random_state=30)

In [ ]:
# Now we're going to build the model

# Starting Linear Regression solution
# Initializing Object, defining param grid and then starting crossvalidation
# After that the training data will be fit and the code will print out best score and best params.
# This is the flowchart that will be followed with all the Regressor and Random Forest solutions
linear_r = LinearRegression()

# Fitting Data
linear_r.fit(X_train, y_train)

In [ ]:
# Initiating Y Prediction
y_prediction_linear = linear_r.predict(X_test)

# MSE calculation
mean_squared_error(y_test, y_prediction_linear)



In [ ]:
# Starting our Random Forest Regressor solution
RF = RandomForestRegressor(random_state=10)
param_RF = {'max_depth' : [5, 10, 15], 'n_estimators': [100, 200, 600], 'min_samples_split': [3, 5, 10]}
RF_cv = GridSearchCV(RF, param_RF, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
RF_cv.fit(X_train, y_train)

In [ ]:
# Getting the best score with Random Forest Regressor
np.sqrt(-1 * RF_cv.best_score_)

In [ ]:
# Best Parameters for Random Forest Regressor
RF_cv.best_params_

In [ ]:
# Staring our XGB Regressor solution
XGB = XGBRegressor(random_state=10)

In [ ]:
# Creating GB params
param_GB ={'max_depth': [3],'n_estimators': [300],'learning_rate': [0.05, 0.1, 0.2], 'min_child_weight': [1,2,3], 'gamma': [0, 0.1, 0.2], 'subsample': [0.8, 0.9, 1.0], 'colsample_bytree': [0.8, 0.9, 1.0],}

# Running GB CV
GB_cv = GridSearchCV(XGB, param_GB, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
GB_cv.fit(X_train, y_train)



In [ ]:
# Print out best GB score
np.sqrt(-1 * GB_cv.best_score_)

In [ ]:
# print out best params for GB
GB_cv.best_params_

In [ ]:
# Starting Gradient Boosting Regressor
GBR = GradientBoostingRegressor()
param_GBR = {'max_depth' : [15, 20, 25], 'n_estimators': [100, 500, 1200], 'min_samples_leaf': [10, 30, 50],'max_features': [0.01, 0.1, 0.7] ,'learning_rate': [0.001, 0.01, 0.1]}

GBR_cv = GridSearchCV(GBR, param_GBR, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
GBR_cv.fit(X_train, y_train)

In [ ]:
# Print out best GBR Score 
np.sqrt(-1 * GBR_cv.best_score_)


In [ ]:
# Print out best GBR Parameters 
GBR_cv.best_params_

In [ ]:
# LGBM Regressor solution
#(scrapped)
#LGBM = lgb.LGBMRegressor()
#param_LGBM = {'boosting_type': ['dart', 'gbdt'], 'num_leaves': [20, 25, 30], 'learning_rate': [0.01, 0.05, 0.1], 'n_estimators': [100, 200, 300], 'verbose' : [-1] }

#LGBM_cv = GridSearchCV(LGBM, param_LGBM,cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
#LGBM_cv.fit(X_train, y_train)


In [ ]:
# Printing out best score
#np.sqrt(-1 * cat_cv.best_score_)

In [ ]:
# Next we're going to build out a Voting Regressor and feed our regressors into it for (hopefully) a better output

voting_r = VotingRegressor([('GBR', GBR_cv.best_estimator_), ('XGB', GB_cv.best_estimator_), ('RFR', RF_cv.best_estimator_)], weights=[1,2,3])
voting_r.fit(X_train, y_train)

In [ ]:
y_prediction_vr = voting_r.predict(X_test)
mean_squared_error(y_test, y_prediction_vr, squared=False)


In [ ]:
# Building an estimator regressor

stimmies = [('gbr', GBR_cv.best_estimator_), ('xgb', GB_cv.best_estimator_), ('rfr', RF_cv.best_estimator_)]
stacks = StackingRegressor(estimators=stimmies, final_estimator=voting_r)

stacks.fit(X_train, y_train)
y_prediction_stacks = stacks.predict(X_test)

# MSE of our Stack regressor
mean_squared_error(y_test, y_prediction_stacks, squared=False)

In [ ]:
# Now We will run the test data through our stack regressor model

# Preprocessing our test data
test_dataframe_pre = preprocess_pipeline.transform(test_dataframe)

# Starting prediction and submitting data
y_stack = np.exp(stacks.predict(test_dataframe_pre))
y_dataframe_out = test_dataframe[['Id']]
y_dataframe_out['SalePrice'] = y_stack
y_dataframe_out.to_csv('results.csv', index=False)




